# Simulation using the `pulser.sampler` module

This notebook is a work document to investigate how to use the sampler correctly in the simulation module.

In [1]:
import numpy as np

import pulser
import pulser_simulation
from pulser import Pulse
from pulser.devices import MockDevice
from pulser.waveforms import BlackmanWaveform, RampWaveform

### Simple sequence

Lets simulate a simple sequence, using either the `simulation.Simulation._extract_sample()` or the `sampler.sample()` functions for samples extraction.

In [2]:
reg = pulser.Register.square(1)
seq = pulser.Sequence(reg, MockDevice)

seq.declare_channel("main", "rydberg_global")

seq.add(Pulse.ConstantPulse(500, 10, 20, 0), "main")

In [3]:
sim = pulser_simulation.Simulation(seq)
res = sim.run()
print(len(sim.samples["Global"]["ground-rydberg"]["amp"]))
fs1 = res.get_final_state()
print(fs1)

501
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.17230094+0.22978882j]
 [0.95786715+0.j        ]]


In [4]:
config2 = pulser_simulation.SimConfig(use_sampler=True)
sim2 = pulser_simulation.Simulation(seq, config=config2)
res2 = sim2.run()
print(len(sim2.samples["Global"]["ground-rydberg"]["amp"]))
fs2 = res2.get_final_state()
print(fs2)

501
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.17230094+0.22978882j]
 [0.95786715+0.j        ]]


The final states are exactly the same.

In [5]:
fs1 == fs2

True

It is no surprise, since we can check that the samples are the same.

In [6]:
def check(a, b):
    np.testing.assert_array_equal(a, b)

check(sim.samples["Global"]["ground-rydberg"]["amp"], sim2.samples["Global"]["ground-rydberg"]["amp"])
check(sim.samples["Global"]["ground-rydberg"]["det"], sim2.samples["Global"]["ground-rydberg"]["det"])
check(sim.samples["Global"]["ground-rydberg"]["phase"], sim2.samples["Global"]["ground-rydberg"]["phase"])


We retain the additional 0 sample on all the parameters, thanks to the `additional_final_sample` flag when calling `sampler.sample`.

In [7]:
print(sim.samples["Global"]["ground-rydberg"]["amp"][-10:])
print(sim2.samples["Global"]["ground-rydberg"]["amp"][-10:])

[10. 10. 10. 10. 10. 10. 10. 10. 10.  0.]
[10. 10. 10. 10. 10. 10. 10. 10. 10.  0.]


### More intricate sequence

In [8]:
reg = pulser.Register.rectangle(1, 2, spacing=10.0, prefix="q")
seq = pulser.Sequence(reg, MockDevice)
seq.declare_channel("main", "rydberg_global")
seq.declare_channel("local", "rydberg_local", initial_target="q0")
seq.add(Pulse.ConstantDetuning(BlackmanWaveform(500, np.pi / 2), 0, 0), "main")
seq.add(Pulse.ConstantPulse(100, 2, -3, 0), "local")
seq.target("q1", "local")
seq.add(Pulse.ConstantAmplitude(4, RampWaveform(123, -20, 31), 0), "local")

seq.add(Pulse.ConstantAmplitude(3, BlackmanWaveform(500, np.pi), 0), "main")


sim3 = pulser_simulation.Simulation(seq)
res3 = sim3.run()

sim4 = pulser_simulation.Simulation(
    seq, config=pulser_simulation.SimConfig(use_sampler=True)
)
res4 = sim4.run()


In [9]:
res3.get_final_state() == res4.get_final_state()

True